Halite Code Version 1

Basic code from YouTuber Sentdex

Includes: Direction choices toward most halite on map and collision detection

Known problems: Large number of ships can get clogged up when attempting to return to shipyard

Solutions: Create dropoffs in various locations to reduce number of ships in one area or code method to allow ships to swap positions to drop off and avoid collisions

In [ ]:
#!/usr/bin/env python3

# Import the Halite SDK, which will let you interact with the game.
import hlt
from hlt import constants
from hlt.positionals import Direction
import random
import logging


# This game object contains the initial game state.
game = hlt.Game()
# Respond with your name.
game.ready("djwangBotv1")
ship_status = {}

while True:
    # Get the latest game state.
    game.update_frame()
    # You extract player metadata and the updated map metadata here for convenience.
    me = game.me
    game_map = game.game_map

    # A command queue holds all the commands you will run this turn.
    command_queue = []
    direction_order = [Direction.North, Direction.South, Direction.East, Direction.West, Direction.Still]
    positionChoices = []
    positions = []

    
    for ship in me.get_ships():
        if ship.id not in ship_status:
            ship_status[ship.id] = "collect"
    
        choices = ship.position.get_surrounding_cardinals() + [ship.position]
        
        positionDict = {}
        haliteDict = {}
        
        for n, direction in enumerate(direction_order):
            positionDict[direction] = choices[n]
            
        for direction in positionDict:
            position = positionDict[direction]
            haliteAmount = game_map[position].halite_amount
            if positionDict[direction] not in positionChoices:
                if direction == Direction.Still:
                    haliteDict[direction] = haliteAmount * 3
                else:
                    haliteDict[direction] = haliteAmount
    
       

       # if ship_status[ship.id] == "returning":
        #    if ship.position == me.shipyard.position:
         #       ship_status[ship.id] = "exploring"
          #  else:
           #     move = game_map.naive_navigate(ship, me.shipyard.position)
            #    command_queue.append(ship.move(move))
             #   continue
       # elif ship.halite_amount >= constants.MAX_HALITE / 4:
        #    ship_status[ship.id] = "returning"
    
        # For each of your ships, move randomly if the ship is on a low halite location or the ship is full.
        #   Else, collect halite.
        logging.info("Ship {} has {} halite.".format(ship.id, ship.halite_amount))
        
        if ship_status[ship.id] == "return":
            move = game_map.naive_navigate(ship, me.shipyard.position)
            positionChoices.append(positionDict[move])
            command_queue.append(ship.move(move))
            
            if move == Direction.Still:
                ship_status[ship.id] = "collect"
            
        elif ship_status[ship.id] == "collect":
            directionalChoice = max(haliteDict, key = haliteDict.get)
            positionChoices.append(positionDict[directionalChoice])
            command_queue.append(ship.move(game_map.naive_navigate(ship, positionDict[directionalChoice])))
            
            if ship.halite_amount > constants.MAX_HALITE * 0.90:
                ship_status[ship.id] = "return"


    # If you're on the first turn and have enough halite, spawn a ship.
    # Don't spawn a ship if you currently have a ship at port, though.
    if game.turn_number <= 100 and me.halite_amount >= constants.SHIP_COST and not game_map[me.shipyard].is_occupied:
        command_queue.append(game.me.shipyard.spawn())

    # Send your moves back to the game environment, ending this turn.
    game.end_turn(command_queue)
